# Imports & Setup

In [1]:
import datetime as dt
import time
import random
import logging
import numpy as np
from collections import defaultdict
from collections import deque
import predict

from optibook.synchronous_client import Exchange
from libs import print_positions_and_pnl, round_down_to_tick, round_up_to_tick

from IPython.display import clear_output

logging.getLogger('client').setLevel('ERROR')

# Function definitions

In [2]:
def insert_quotes(exchange, instrument, bid_price, ask_price, bid_volume, ask_volume):
    if bid_volume > 0:
        # Insert new bid limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='ioc',
        )
        
        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)

    if ask_volume > 0:
        # Insert new ask limit order on the market
        exchange.insert_order(
            instrument_id=instrument.instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='ioc',
        )

        # Wait for some time to avoid breaching the exchange frequency limit
        time.sleep(0.05)
        
def is_liquid(bid_prices, ask_prices, trading_volume_bids, trading_volume_asks, liquidity_threshold=0.1, unbalance_threshold=0.2):
    # Calculate the bid-ask spread as the difference between ask and bid prices
    bid_ask_spread = [ask - bid for ask, bid in zip(ask_prices, bid_prices)]

    # Calculate the average bid-ask spread
    average_spread = np.mean(bid_ask_spread)

    # Calculate the total buying and selling volumes
    total_bid_volume = sum(trading_volume_bids)
    total_ask_volume = sum(trading_volume_asks)

    # Calculate the spread-to-volume ratio
    spread_to_volume = average_spread / ((np.mean(bid_prices) + np.mean(ask_prices)) / 2)

    # Calculate the order imbalance
    order_imbalance = abs(total_bid_volume - total_ask_volume) / (total_bid_volume + total_ask_volume)

    # Check for market liquidity based on the liquidity threshold
    is_liquid = spread_to_volume <= liquidity_threshold

    # Check for market unbalance based on the unbalance threshold
    is_unbalanced = order_imbalance >= unbalance_threshold

    print('is_liquid %s, is_unbalanced %s',is_liquid, is_unbalanced)
    return is_liquid, is_unbalanced

    
    

In [3]:
def calculate_direction(rsi_indicator,bid_ask_indicator,sentiment_score,last_tweet_dict,instrument_id):
    
    print('last_tweet_dict %s',last_tweet_dict)
    
    iteration_number = 0
    
    if instrument_id in last_tweet_dict.keys():
        sentiment_score, iteration_number = last_tweet_dict[instrument_id]
    
    if (iteration_number>0):
        last_tweet_dict[instrument_id] = [sentiment_score,iteration_number-1]
    else:
        sentiment_score = 0.5
    
    # Sample weights for each factor (you should adjust these based on relevance)
    weight_sentiment = 0.3
    weight_volume = 0.3
    weight_rsi = 0.4
    
    if sentiment_score > 0.5:
        ss2 = 1
        weight_sentiment = 0.8
        weight_volume = 0.1
        weight_rsi = 0.1
    elif sentiment_score < 0.5:
        ss2=0
        weight_sentiment = 0.8
        weight_volume = 0.1
        weight_rsi = 0.1
    else:
        ss2 = 0.5

    
    # Calculate the weighted sum to determine the direction
    weighted_sum = (
        rsi_indicator * weight_rsi +
        bid_ask_indicator * weight_volume +
        ss2 * weight_sentiment

        # sentiment_score * weight_sentiment
    )
    
    print('sentiment score %s', sentiment_score)
    print('volume_difference score %s', bid_ask_indicator)
    print('market_indicators (rsi) score %s',rsi_indicator)

    # Define a threshold for the gradient (you can adjust this based on your criteria)
    gradient_threshold = 0.5  # Example threshold: 0.6 (neutral position)

    print('weighted_sum gradient %s',weighted_sum)
    # Estimate market maker's position direction based on the gradient
    if weighted_sum > gradient_threshold:
        direction = "Long"  # Positive gradient
    elif weighted_sum < gradient_threshold:
        direction = "Short"  # Negative gradient
    else:
        direction = "Neutral"  # Neutral gradient

    print(f"Market Maker Position Direction: {direction}")
    
    return direction, weighted_sum


In [4]:
def sliding_window(rolling_window, current_price):
    
    # Add the new element to the right side of the deque
    rolling_window.append(current_price)
    
    return rolling_window

In [5]:
def calculate_ema(data, period):
    # if len(data) < 2:
    # return data  # Not enough data to calculate EMA, return the data as is

    if len(data) < period:
        return sum(data) / len(data)  # Calculate and return the simple average

    multiplier = 2 / (period + 1)
    ema = [data[0]]  # Initialize EMA with the first data point

    for i in range(1, len(data)):
        # Calculate EMA for the current data point
        ema_value = (data[i] - ema[-1]) * multiplier + ema[-1]
        ema.append(ema_value)

    return ema

In [6]:
def calculate_rsi(data, period):
    # if len(data) < 2:
    #     return 0.5  # Return a neutral value if there isn't enough data

    print('data %s',len(data))
    if len(data) < period:
        return 0 #sum(data)/len(data)/  # Return a neutral value if there isn't enough data for the specified period

    gains = []
    losses = []

    for i in range(1, len(data)):
        price_difference = data[i] - data[i - 1]
        if price_difference >= 0:
            gains.append(price_difference)
            losses.append(0)
        else:
            gains.append(0)
            losses.append(abs(price_difference))

    average_gain = sum(gains[:period]) / period
    average_loss = sum(losses[:period]) / period

    if average_loss == 0:
        return 1.0  # Handle the case where there are no losses

    relative_strength = average_gain / average_loss
    rsi = 100 - (100 / (1 + relative_strength))
    return rsi / 100  # Normalize RSI to a value between 0 and 1

# # Example data and period
# data = [23.45, 24.12, 25.67, 26.54, 27.36, 28.45, 29.18, 30.27, 31.42, 32.11, 33.45, 34.23]
# period = 14

# current_price = 35.0  # Replace with the current price you want to compare

# rsi_value = calculate_rsi(data, period)
# print("RSI Value:", rsi_value)


In [7]:
def calculate_bid_ask_indicator(bid_volume, ask_volume):
    # Ensure bid_volume and ask_volume are non-negative
    bid_volume = max(0, bid_volume)
    ask_volume = max(0, ask_volume)

    # Calculate the bid-ask indicator
    total_volume = bid_volume + ask_volume
    if total_volume == 0:
        return 0.5  # Default to 0.5 if total volume is zero (neutral)

    bid_ask_indicator = bid_volume / (bid_volume + ask_volume)
    
    # Ensure the result is between 0 and 1
    return max(0, min(1, bid_ask_indicator))

In [8]:
def get_tweet_impact_score(feed,instrument_id,last_tweet_dict):
    
    print('feed %s',feed)
    # last_tweet_dict[instrument_id] = "tweet[1]"

    if len(feed)==0:
        return 0.5
    if not feed[0]:
        predict.register(feed[0].post)
        return 0.5
    tweet = predict.get()
    
    print('tweet %s',tweet)

    print('feed[0] %s',feed[0])
    time.sleep(30)

    if tweet is not None:
        print('tweet %s',tweet)
        time.sleep(20)

        # if tweet[0] == instrument_id:
        

        last_tweet_dict[tweet[0]] = tweet
        # score range [0,1]
        return (tweet[1]+1)/2
    
    # if tweet don't exist, it's neutral
    
    return 0.5

In [9]:
def get_tweet_impact_score(feed,instrument_id,last_tweet_dict):
    if len(feed)>0:
        tweet = feed[0].post
        print("tweet --" + tweet)
        prediction = predict.predict(tweet)
        
        if prediction is not None:
            if prediction[1] is not None:
                print("impact -- " +str(prediction[1]))

                last_tweet_dict[prediction[0]] = [(prediction[1]+1)/2 ,23]
                # score range [0,1]
                return (prediction[1]+1)/2    
    else:
        return 0.5

# Main algorithm

In [ ]:
exchange = Exchange()
exchange.connect()

INSTRUMENTS = exchange.get_instruments()

QUOTED_VOLUME = 3
FIXED_MINIMUM_CREDIT = 0.1
PRICE_RETREAT_PER_LOT = 0.005
POSITION_LIMIT = 100
MICHAEL_JORDAN_WINDOW = 223
MICHAEL_JORDAN_TWEET_ITERATIONS = 223

queue_stock = defaultdict(lambda: deque(maxlen=MICHAEL_JORDAN_WINDOW))
last_tweet_dict = defaultdict()

while True:
    print(f'')
    print(f'-----------------------------------------------------------------')
    print(f'TRADE LOOP ITERATION ENTERED AT {str(dt.datetime.now()):18s} UTC.')
    print(f'-----------------------------------------------------------------')

    # Display our own current positions in all stocks, and our PnL so far
    print_positions_and_pnl(exchange)
    print(f'')
    print(f'          (ourbid) mktbid :: mktask (ourask)')
    
    

    for instrument in INSTRUMENTS.values():
        # Remove all existing (still) outstanding limit orders
        # exchange.delete_orders(instrument.instrument_id)
    
        tweet_impact = get_tweet_impact_score(exchange.poll_new_social_media_feeds(),instrument.instrument_id,last_tweet_dict)

        # Obtain order book and only skip this instrument if there are no bids or offers available at all on that instrument,
        # as we we want to use the mid price to determine our own quoted price
        instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
        
        # print('instrument_order_book %s',instrument_order_book)
        if not (instrument_order_book and instrument_order_book.bids and instrument_order_book.asks):

            print(f'{instrument.instrument_id:>6s} --     INCOMPLETE ORDER BOOK')
            continue

        bids = [bid.price for bid in instrument_order_book.bids]
        asks = [bid.price for bid in instrument_order_book.asks]
        volume_bids = [bid.volume for bid in instrument_order_book.bids]
        volume_asks = [bid.volume for bid in instrument_order_book.asks]
        
        volume_bids_sum = sum(volume_bids)
        volume_asks_sum = sum(volume_asks)
        
        volume_difference = volume_bids_sum-volume_asks_sum

        
        mid_price = (bids[0]+asks[0])/2
        
        print('mid price %s',mid_price)

        sliding_window_var = sliding_window(queue_stock[instrument.instrument_id],mid_price)
        
        

        # ema = calculate_ema(sliding_window_var,10)
        rsi = calculate_rsi(sliding_window_var,MICHAEL_JORDAN_WINDOW)
        
        bid_ask_indicator = calculate_bid_ask_indicator(volume_bids_sum,volume_asks_sum)
        
        direction,gradient = calculate_direction(rsi,bid_ask_indicator,tweet_impact,last_tweet_dict,instrument.instrument_id)
        print('direction %s',direction)
        liquid, unbalanced = is_liquid(bids, asks, volume_bids, volume_asks, liquidity_threshold=0.1, unbalance_threshold=0.2)
        
        if liquid and not unbalanced:
            # Obtain own current position in instrument
            position = exchange.get_positions()[instrument.instrument_id]

            # Obtain best bid and ask prices from order book to determine mid price
            best_bid_price = instrument_order_book.bids[0].price
            best_ask_price = instrument_order_book.asks[0].price
            mid_price = (best_bid_price + best_ask_price) / 2.0 

            # Calculate our fair/theoretical price based on the market mid price and our current position
            theoretical_price = mid_price #- PRICE_RETREAT_PER_LOT * position

            # Calculate final bid and ask prices to insert
            # bid_price = round_down_to_tick(theoretical_price - FIXED_MINIMUM_CREDIT, instrument.tick_size)
            # ask_price = round_up_to_tick(theoretical_price + FIXED_MINIMUM_CREDIT, instrument.tick_size)
            print('outstanding order 1 %s',exchange.get_outstanding_orders(instrument.instrument_id))

            # print('get trade history %s',exchange.get_trade_history(instrument.instrument_id))
            print(exchange.get_positions_and_cash())
            if direction == 'Long':
                
                bid_price = mid_price+ FIXED_MINIMUM_CREDIT
                ask_price = mid_price + 2*FIXED_MINIMUM_CREDIT

                # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
                max_volume_to_buy = POSITION_LIMIT - position
                max_volume_to_sell = POSITION_LIMIT + position

                # bid_volume = min(int(10*gradient),  max_volume_to_buy)
                # ask_volume = min(int(10*gradient), max_volume_to_sell)
                bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
                ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)
                
                
                if instrument.instrument_id in last_tweet_dict.keys():
                    impact,iteration_nr = last_tweet_dict[instrument.instrument_id]
                    if iteration_nr>0:
                        bid_price = mid_price*(1+(impact*2-1))
                        ask_price = mid_price*(1+(impact*2-1))+FIXED_MINIMUM_CREDIT
                        


                # Display information for tracking the algorithm's actions
                print(f'{instrument.instrument_id:>6s} -- ({bid_price:>6.2f}) {best_bid_price:>6.2f} :: {best_ask_price:>6.2f} ({ask_price:>6.2f})')

                print('order in long %s',instrument, bid_price, ask_price, bid_volume, ask_volume)
                # Insert new quotes
                # try:
                insert_quotes(exchange, instrument, round(bid_price,1), round(ask_price,1), bid_volume, ask_volume)
                # except:
                #     print('self trade lol')
            if direction == 'Short':
                bid_price = mid_price - FIXED_MINIMUM_CREDIT
                ask_price = mid_price - 2*FIXED_MINIMUM_CREDIT

                if instrument.instrument_id in last_tweet_dict.keys():
                    impact,iteration_nr = last_tweet_dict[instrument.instrument_id]
                    if iteration_nr>0:
                        bid_price = mid_price*(1+(impact*2-1))
                        ask_price = mid_price*(1+(impact*2-1))-FIXED_MINIMUM_CREDIT
                        
                # Calculate bid and ask volumes to insert, taking into account the exchange position_limit
                max_volume_to_buy = POSITION_LIMIT - position
                max_volume_to_sell = POSITION_LIMIT + position

                # bid_volume = min(int(10*gradient),  max_volume_to_buy)
                # ask_volume = min(int(10*gradient), max_volume_to_sell)
                bid_volume = min(QUOTED_VOLUME, max_volume_to_buy)
                ask_volume = min(QUOTED_VOLUME, max_volume_to_sell)
                
                # Display information for tracking the algorithm's actions
                print(f'{instrument.instrument_id:>6s} -- ({bid_price:>6.2f}) {best_bid_price:>6.2f} :: {best_ask_price:>6.2f} ({ask_price:>6.2f})')
                print('order in short %s',instrument, bid_price, ask_price, bid_volume, ask_volume)

                # Insert new quotes
                # try:
                insert_quotes(exchange, instrument, round(bid_price,1), round(ask_price,1), bid_volume, ask_volume)
                # except:
                #     print('self trade lol')
            else :
                print('this is neutral lololololo')
        else:
            print('The market is illiquid or unbalanced')
            ## code bellow performed bad
#             positions = exchange.get_positions()

#             pos = positions[instrument.instrument_id]
#             if pos > 0:
        
#                 instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
#                 MIN_SELLING_PRICE = instrument_order_book.bids[0].price

#                 exchange.insert_order(instrument.instrument_id, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')

#             if pos < 0:
#                 instrument_order_book = exchange.get_last_price_book(instrument.instrument_id)
#                 MIN_BUYING_PRICE = instrument_order_book.asks[0].price

#                 exchange.insert_order(instrument.instrument_id, price=MIN_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
            # print(f'Positions before: {positions}')
            # print(f'\nPnL before: {pnl:.2f}')

                # Wait for a few seconds to refresh the quotes
            print(f'\nWaiting for 2 seconds.')
            time.sleep(0.05)
    
    # Clear the displayed information after waiting
    clear_output(wait=True)


-----------------------------------------------------------------
TRADE LOOP ITERATION ENTERED AT 2023-09-17 04:26:52.651643 UTC.
-----------------------------------------------------------------
Positions:
  NVDA      :  -31
  ING       : -100
  SAN       : -100
  PFE       :  -61
  CSCO      :  -43

PnL: -20777.40

          (ourbid) mktbid :: mktask (ourask)
mid price %s 52.0
data %s 223
last_tweet_dict %s defaultdict(None, {'NVDA': [0.5177531000226736, 0], 'SAN': [0.5182078201323748, 0], 'PFE': [0.5162079092115164, 0]})
sentiment score %s 0.5
volume_difference score %s 0.5
market_indicators (rsi) score %s 0.49773755656108576
weighted_sum gradient %s 0.4990950226244343
Market Maker Position Direction: Short
direction %s Short
is_liquid %s, is_unbalanced %s True False
outstanding order 1 %s {}
{'NVDA': {'volume': -31, 'cash': -1905.0000000000223}, 'ING': {'volume': -100, 'cash': -3458.799999999991}, 'SAN': {'volume': -100, 'cash': 3191.999999999967}, 'PFE': {'volume': -61, 'cash': -

2023-09-17 04:26:53,232 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443


this is neutral lololololo
mid price %s 63.8
data %s 223
last_tweet_dict %s defaultdict(None, {'NVDA': [0.5177531000226736, 0], 'SAN': [0.5182078201323748, 0], 'PFE': [0.5162079092115164, 0]})
sentiment score %s 0.5
volume_difference score %s 0.5879120879120879
market_indicators (rsi) score %s 0.4131054131054131
weighted_sum gradient %s 0.4916157916157916
Market Maker Position Direction: Short
direction %s Short
is_liquid %s, is_unbalanced %s True False
outstanding order 1 %s {}
{'NVDA': {'volume': -31, 'cash': -1905.0000000000223}, 'ING': {'volume': -100, 'cash': -3458.799999999991}, 'SAN': {'volume': -100, 'cash': 3191.999999999967}, 'PFE': {'volume': -58, 'cash': -2305.7999999999756}, 'CSCO': {'volume': -46, 'cash': 684.9999999999635}}
   SAN -- ( 63.70)  63.60 ::  64.00 ( 63.60)
order in short %s Instrument(instrument_id=SAN, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=50.0000, relative_change=50.00%), instrument_type=InstrumentType.STOCK, instrument_group=SA

2023-09-17 04:26:53,357 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
2023-09-17 04:26:53,373 [urllib3.connectionpool] [MainThread  ] Starting new HTTPS connection (1): huggingface.co:443
2023-09-17 04:26:53,492 [urllib3.connectionpool] [MainThread  ] https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/pytorch_model.bin HTTP/1.1" 302 0
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSeque